In [0]:
import pandas as pd, numpy as np
import datetime as dt

In [0]:
### Load original attribute labelled with Fuzzy Match (Un-normalized), multiple attributes need to be delimited
attr_path = '/dbfs/FileStore/tables/DATA_SCIENCE/COLOR/'
attributed_df = pd.read_csv(attr_path +  'COLOR_Floral_12192023_DPT.csv', dtype={"GTIN_NO": 'int64'}, low_memory=False)
attrib_field = 'EXTRACTED_COLOR'
delim = '|'
attributed_df = attributed_df[attributed_df[attrib_field] != 'OTHER']

In [0]:
### Replace 'OTHER' with current attribute value if available
attributed_df['COLOR_CURRENT'] = attributed_df.COLOR_CURRENT.fillna('NONE')
attributed_df['EXTRACTED_COLOR'] = attributed_df.\
    apply(lambda x: x['COLOR_CURRENT'] if x['COLOR_CURRENT'] != 'NONE' else x['EXTRACTED_COLOR'], axis = 1)

In [0]:
single_attributes = pd.Series(attributed_df[attrib_field].apply(lambda x: x.split(delim)).sum())

In [0]:
### Cumulative Percent Prevalence of Single Attributes ranked from top
attrib_cum_counts = pd.DataFrame(100* single_attributes.value_counts().cumsum()/len(single_attributes), columns = ['PERCENT_CUMULATIVE_COUNT'])
cutoff = 100 # Select top attributes that add upto this percentage of all non-other attributed items)
'''attrib_cum_counts[attrib_cum_counts.PERCENT_CUMULATIVE_COUNT <= cutoff][:100]\
    .reset_index(drop = False).rename(columns = {'index':attrib_field}).to_csv(attr_path+'Normalized_COLOR_FLORAL_Top100.csv', index =None)'''

"attrib_cum_counts[attrib_cum_counts.PERCENT_CUMULATIVE_COUNT <= cutoff][:100]    .reset_index(drop = False).rename(columns = {'index':attrib_field}).to_csv(attr_path+'Normalized_COLOR_FLORAL_Top100.csv', index =None)"

In [0]:
### Cumulative Percent Prevalence of Single Attributes ranked from top
### Cumulative Percent Prevalence of Single Attributes ranked from top
attrib_cum_counts[attrib_cum_counts.PERCENT_CUMULATIVE_COUNT <= cutoff][:100]\
    .reset_index(drop = False).rename(columns = {'index':attrib_field}).to_csv(attr_path+'Normalized_COLORS_Floral.csv', index =None)

In [0]:
normalized_list = list(attrib_cum_counts[attrib_cum_counts.PERCENT_CUMULATIVE_COUNT <= cutoff][:100].index)

In [0]:
sorted(normalized_list)

['ANTIQUE',
 'APPLE',
 'ASSORTED',
 'ASSORTED COLORS',
 'AZURE',
 'BARK',
 'BEIGE',
 'BERRY',
 'BI-COLOR',
 'BIRCH',
 'BLACK',
 'BLUE',
 'BRONZE',
 'BROWN',
 'BURGUNDY',
 'BURLAP',
 'CAYENNE',
 'CHAMPAGNE',
 'CHOCOLATE',
 'CLASSIC WHITE',
 'CLAY',
 'CLEAR',
 'COCOA',
 'COFFEE',
 'CONCRETE',
 'COPPER',
 'CORAL',
 'CREAM',
 'CRUSH',
 'DARK',
 'DARK GREEN',
 'DARK PINK',
 'DARK RED',
 'DEEP',
 'FROST',
 'FUCHSIA',
 'GOLD',
 'GOLDEN',
 'GRAY',
 'GREEN',
 'GREY',
 'HAPPY BIRTHDAY',
 'HEIRLOOM',
 'HIBISCUS',
 'HOT PINK',
 'IRON',
 'IVORY',
 'LAVENDER',
 'LEMON',
 'LIGHT',
 'LIGHT PINK',
 'LIME',
 'LOVE',
 'MAHOGANY',
 'MARIGOLD',
 'MEDIUM',
 'MELON',
 'MULTI COLOR',
 'MULTI-COLORED',
 'NATURAL',
 'NEON',
 'OLIVE',
 'ORANGE',
 'ORCHID',
 'OTHER',
 'PEACOCK',
 'PEARL',
 'PEONY',
 'PINEAPPLE',
 'PINK',
 'PLUM',
 'PRIMROSE',
 'PURPLE',
 'PURPLE RAIN',
 'RAINBOW',
 'RASPBERRY',
 'RED',
 'ROSE',
 'ROSE GOLD',
 'RUST',
 'SAGE',
 'SALMON',
 'SAPPHIRE',
 'SCARLET',
 'SILVER',
 'STEEL',
 'STRAWBERRY',

In [0]:
### Incorporating PO Feedback
normalized_list = list(attrib_cum_counts[attrib_cum_counts.PERCENT_CUMULATIVE_COUNT <= cutoff][:100].index)
discard =['ANTIQUE','APPLE','BARK','BERRY','BI-COLOR','BRONZE','CAYENNE','CHAMPAGNE', \
'CHOCOLATE','CLAY','COCOA', 'COFFEE','CONCRETE', \
'COPPER','CORAL','CREAM','CRUSH','DARK','DEEP','FROST','HAPPY BIRTHDAY','HEIRLOOM', \
'IRON', 'LIGHT','LOVE','MARIGOLD', 'MEDIUM','MELON','NATURAL','ORCHID','PEACOCK', 'PEARL', \
'PEONY','PINEAPPLE','SAGE','STEEL','STRIPED','URBAN','VALENTINE','VARIETY',\
'WATERMELON','WOOD','ZINNIA']
normalized_list = [ i for i in normalized_list if i not in discard] + ['OTHER']
rename = {'ASSORTED':'MULTICOLOR','ASSORTED COLORS':'MULTICOLOR', 'CLASSIC WHITE':'WHITE', \
          'MULTI-COLORED':'MULTICOLOR','MULTI COLOR':'MULTICOLOR',\
          'GREY':'GRAY','DARK GRAY':'GRAY','LIGHT GRAY':'GRAY', \
          'NAVY BLUE':'NAVY','DARK BLUE' : 'BLUE', 'LIGHT BLUE': 'BLUE', \
          'DARK BROWN' : 'BROWN', 'LIGHT BROWN' : 'BROWN', \
          'DARK GREEN' :  'GREEN', 'LIGHT GREEN' :  'GREEN','GOLDEN':'GOLD', \
          'LIGHT PINK':'PINK', 'DARK PINK':'PINK', 'DARK RED': 'RED'}
normalized_list += rename.values()

def normalize_colors(attr_string, normalized_list = normalized_list ,delim_read = '|', delim_write = ';',rename = rename):
    attr_list = attr_string.split(delim_read)
    attr_list = [rename[i] if i in rename.keys() else i for i in attr_list]
    attr_list = [i for i in attr_list if i in normalized_list]
    if len(attr_list) > 0:
        if( len(attr_list) == 1 ) & (attr_list[0]== 'STRIPED'): return 'OTHER'
        return (delim_write).join(attr_list)
    else: return 'OTHER'

In [0]:
'''Add a flag to indicate marketplace items
Query used to generate the list:
"SELECT  GTIN_NO, SLS_CHA_VLU
FROM CURIDQ.pimmart.ITEM_VIEW"
'''
mkt_flg = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/MARKETPLACE_FLAG_120723.csv')
attributed_df = attributed_df.merge(mkt_flg, how = 'left', on = 'GTIN_NO')

In [0]:
### Write normalized attributes in semicolon-separated format
pd.options.mode.chained_assignment = None
cf_dpt = 'Floral' # CFIC Department name
attributed_df['NORMALIZED_COLOR'] = attributed_df.EXTRACTED_COLOR.apply(lambda x: normalize_colors(x))
attributed_df['GTIN_NO'] = attributed_df.GTIN_NO.astype(str).apply(lambda x:(14- len(x))*'0' + x )
display_cols = ['GTIN_NO','VND_ECOM_DSC','ITM_LVL_DSC','TAG_DSC','COLOR_CURRENT','NORMALIZED_COLOR','SLS_CHA_VLU']
today = dt.datetime.today().strftime("%m%d%Y")
attributed_df[display_cols].drop_duplicates('GTIN_NO').to_csv(attr_path +'COLOR_NORMALIZED_'+\
     cf_dpt.replace(' ','_').replace('-','_') + '_' + today + '.csv', index = None)

In [0]:
### Please create this config file only once the normalized list is validated, sanitized and  finalized
# Store Item group (DPT/ COM) specific configs in config - file
import json
config_path = '/dbfs/FileStore/tables/DATA_SCIENCE/FuzzyConfigs/'
attrib = 'COLOR'
me = 'SOPHIA'
configs = {}
configs['CFIC_DPT'] = cf_dpt
configs['Attribute'] = attrib
configs['Normalized_list'] = normalized_list
configs['Substitutions'] = rename
with open(config_path + attrib + '_' +cf_dpt + '_' + me + '_'+ today+ '.json', 'w') as fp:
    json.dump(configs, fp)

In [0]:
#Write normalized list to csv for manual validation guidance
pd.Series(normalized_list).to_csv(attr_path + 'NORMALIZED_LIST_'+attrib + '_' +cf_dpt +  '_'+ today+ '.csv',index = None )

In [0]:
with open('/dbfs/FileStore/tables/DATA_SCIENCE/FuzzyConfigs/COLOR_Floral_SOPHIA_01052024.json') as f:
    d = json.load(f)
    print(d)

{'CFIC_DPT': 'Floral', 'Attribute': 'COLOR', 'Normalized_list': ['RED', 'ROSE', 'WHITE', 'PINK', 'GREEN', 'ASSORTED', 'BLACK', 'YELLOW', 'BLUE', 'PURPLE', 'ORANGE', 'GOLD', 'MULTI-COLORED', 'SILVER', 'CLEAR', 'BROWN', 'LEMON', 'GRAY', 'RAINBOW', 'LAVENDER', 'GOLDEN', 'OTHER', 'BURGUNDY', 'LIME', 'GREY', 'VIOLET', 'BIRCH', 'BURLAP', 'BEIGE', 'HOT PINK', 'IVORY', 'NEON', 'PURPLE RAIN', 'PRIMROSE', 'OLIVE', 'SCARLET', 'LIGHT PINK', 'TERRACOTTA', 'CLASSIC WHITE', 'HIBISCUS', 'STRAWBERRY', 'DARK GREEN', 'DARK RED', 'ASSORTED COLORS', 'AZURE', 'ROSE GOLD', 'RUST', 'RASPBERRY', 'DARK PINK', 'SALMON', 'TEAL', 'SAPPHIRE', 'FUCHSIA', 'TAN', 'PLUM', 'MAHOGANY', 'MULTI COLOR', 'OTHER', 'MULTICOLOR', 'MULTICOLOR', 'WHITE', 'MULTICOLOR', 'MULTICOLOR', 'GRAY', 'GRAY', 'GRAY', 'NAVY', 'BLUE', 'BLUE', 'BROWN', 'BROWN', 'GREEN', 'GREEN', 'GOLD', 'PINK', 'PINK', 'RED'], 'Substitutions': {'ASSORTED': 'MULTICOLOR', 'ASSORTED COLORS': 'MULTICOLOR', 'CLASSIC WHITE': 'WHITE', 'MULTI-COLORED': 'MULTICOLOR', 'M

In [0]:
df = pd.read_csv(attr_path +'COLOR_NORMALIZED_'+\
     cf_dpt.replace(' ','_').replace('-','_') + '_' + today + '.csv')
df.head()     

,GTIN_NO,VND_ECOM_DSC,ITM_LVL_DSC,TAG_DSC,COLOR_CURRENT,NORMALIZED_COLOR,SLS_CHA_VLU
0,685229045683,Amaryllis Red Lion,NaN,GSBC AMARYLLIS RED LION,NONE,RED,Store
1,19518120476,10 Poinsettia Red,Premium Red Poinsettia,GRCC POINSETTIA RED PLAT,NONE,RED,Store
2,22532136293,3.8 Exotic Angel Assorted,NaN,CSTA SUCCLNT EXOTIC ANGEL,NONE,MULTICOLOR,NaN
3,19518117643,"6"" Poinsettia Red",Primrose in Bowl Planter,GRCC POINSETTIA RED PREM,NONE,PRIMROSE;RED,Store
4,22532136378,4.8 Exotic Angel Assorted,NaN,4.8 EXOTIC ANGEL PLANTS,NONE,MULTICOLOR,Store


In [0]:
df1 = pd.read_csv(attr_path + 'NORMALIZED_LIST_'+attrib + '_' +cf_dpt +  '_'+ today+ '.csv')
df1.head()

,0
0,RED
1,ROSE
2,WHITE
3,PINK
4,GREEN
